In [ ]:
import pandas as pd
from pathlib import Path
import os
from collections import Counter
import math

In [ ]:
def select_row(df,row,values):
    if type(values)==list:
        mask = df[row].isin(values)
    elif type(values) in [int,float,str]:
        mask = df[row] == values
    return df[mask]
def get_glide_results(ligand_list):
    if type(ligand_list) in [str]:
        ligand_list=[ligand_list]
    rows = select_row(docking_results,"Title",ligand_list)
    scores = list(rows["docking score"].values )
    return [min(scores)]

In [ ]:
path=Path("/home/selina/docking/Glide/spreadsheet")
glide_results=pd.read_csv(path/"glide_docking_all.csv")

receptor="6zsl_monomer_a"
top_scorer=Path("/home/selina/docking/analysis")
top_scorer=[[str(a),str(b)] for a,b in pd.read_csv(top_scorer/receptor/"top_100_ligands.csv")[["ZINC","ECBD"]].values]

In [ ]:
# glide_results=glide_results[['Title', 'docking score','glide gscore','eos','pubchem','zinc','smiles','formula','Chemistry Notes']]
glide_results=glide_results[['Title', 'docking score','glide gscore','eos','pubchem','zinc','smiles','formula']]
start_id=max(glide_results[glide_results["Title"].apply(lambda x: x.startswith("6zsl"))].index)
all_glide_results=glide_results.loc[start_id+1:].reset_index(drop=True)
docking_results=all_glide_results[~all_glide_results["Title"].isin(["ATP","ADP"])].sort_values("docking score")
all_glide_results.head(3)

In [ ]:
max_index = all_glide_results.groupby(["Title"]).apply(
    lambda x: list(x.index[x["docking score"]!=min(x["docking score"])]))
doubles = {k:v for k,v in Counter(all_glide_results[all_glide_results.duplicated(["Title"],keep=False)]["Title"]).items() 
                  if not str(k) == "nan"}
all_glide_results=all_glide_results.drop([x for xs in [i for i in max_index.values] for x in xs])
all_glide_results=all_glide_results.sort_values("docking score").reset_index(drop=True)
all_glide_results.head(3)

In [ ]:
max_index = docking_results.groupby(["Title"]).apply(
    lambda x: list(x.index[x["docking score"]!=min(x["docking score"])]))
doubles = {k:v for k,v in Counter(docking_results[docking_results.duplicated(["Title"],keep=False)]["Title"]).items() 
                  if not str(k) == "nan"}
docking_results=docking_results.drop([x for xs in [i for i in max_index.values] for x in xs]).reset_index(drop=True)
docking_results.head(3)

In [ ]:
docking_score=pd.DataFrame([t+get_glide_results(t) for t in top_scorer if get_docking_score(t) != []],
                           columns=["ZINC","ECBD","docking score"]
                          ).sort_values("docking score").reset_index(drop=True)
docking_score=docking_score.reset_index()
docking_score.head()

In [ ]:
docking_score.to_csv("./top100_docking_scores.csv",index=False)
all_glide_results.reset_index(drop=False).to_csv("./all_docking_results.csv",index=False)